In [1]:
!ls ../data/shizuoka

L03-b-21_5138-jgd2011_GML      L03-b-21_5239-jgd2011_GML.zip
L03-b-21_5138-jgd2011_GML.zip  L03-b-21_5338-jgd2011_GML
L03-b-21_5237-jgd2011_GML      L03-b-21_5338-jgd2011_GML.zip
L03-b-21_5237-jgd2011_GML.zip  L03-b-21_5339-jgd2011_GML
L03-b-21_5238-jgd2011_GML      L03-b-21_5339-jgd2011_GML.zip
L03-b-21_5238-jgd2011_GML.zip  polygon
L03-b-21_5239-jgd2011_GML


In [2]:
import geopandas as gpd
from glob import glob
import pandas as pd
import numpy as np
import rasterio
from rasterio.features import rasterize

In [3]:
shp_files = glob('../data/shizuoka/*/L03*.shp')
shp_files

['../data/shizuoka/L03-b-21_5338-jgd2011_GML/L03-b-21_5338.shp',
 '../data/shizuoka/L03-b-21_5239-jgd2011_GML/L03-b-21_5239.shp',
 '../data/shizuoka/L03-b-21_5339-jgd2011_GML/L03-b-21_5339.shp',
 '../data/shizuoka/L03-b-21_5237-jgd2011_GML/L03-b-21_5237.shp',
 '../data/shizuoka/L03-b-21_5238-jgd2011_GML/L03-b-21_5238.shp',
 '../data/shizuoka/L03-b-21_5138-jgd2011_GML/L03-b-21_5138.shp']

In [4]:
gdf = gpd.GeoDataFrame()
for shp_file in shp_files:
    tmp = gpd.read_file(shp_file)
    gdf = gpd.GeoDataFrame(pd.concat([gdf, tmp], ignore_index=True))

In [5]:
gdf.head()

,L03b_001,L03b_002,L03b_003,geometry
0,5338500000,0500,20201025,"POLYGON ((138 35.75, 138 35.75083, 138.00125 3..."
1,5338500001,0500,20201025,"POLYGON ((138.00125 35.75, 138.00125 35.75083,..."
2,5338500002,0500,20201025,"POLYGON ((138.0025 35.75, 138.0025 35.75083, 1..."
3,5338500003,0500,20201025,"POLYGON ((138.00375 35.75, 138.00375 35.75083,..."
4,5338500004,0500,20201025,"POLYGON ((138.005 35.75, 138.005 35.75083, 138..."


In [6]:
print(gdf.geometry[0])

POLYGON ((138 35.75000000000006, 138 35.75083333300006, 138.00125000000003 35.75083333300006, 138.00125000000003 35.75000000000006, 138 35.75000000000006))


In [7]:
# 座標系を確認
print(gdf.crs) 

# Webメルカトル（EPSG:3857）に変換
gdf_web_mercator = gdf.to_crs(epsg=3857)

EPSG:6668


In [8]:
print(gdf_web_mercator.crs) 

EPSG:3857


In [9]:
gdf = None

In [10]:
gdf_web_mercator.head()

,L03b_001,L03b_002,L03b_003,geometry
0,5338500000,0500,20201025,"POLYGON ((15362089.729 4266276.06, 15362089.72..."
1,5338500001,0500,20201025,"POLYGON ((15362228.879 4266276.06, 15362228.87..."
2,5338500002,0500,20201025,"POLYGON ((15362368.028 4266276.06, 15362368.02..."
3,5338500003,0500,20201025,"POLYGON ((15362507.178 4266276.06, 15362507.17..."
4,5338500004,0500,20201025,"POLYGON ((15362646.327 4266276.06, 15362646.32..."


In [11]:
# ポリゴンのシェープファイルの読み込み
polygon_file = "../data/shizuoka/polygon/shizuoka.shp"
gdf_polygon = gpd.read_file(polygon_file)

# データの確認
print(gdf_polygon.head()) 

   id                                           geometry
0   1  POLYGON ((15387378.041 4251823.906, 15391391.1...


In [12]:
# 静岡県を切り出す

polygon = gdf_polygon.geometry[0]  # 必要に応じてインデックスを調整

# ポリゴン内にピクセルの緯度経度が入っているかを判定

gdf_web_mercator['is_in'] = np.where(polygon.contains(gdf_web_mercator['geometry'].centroid), 1, 0)

In [13]:
gdf_shizuoka = gdf_web_mercator[gdf_web_mercator['is_in'] == 1]
gdf_shizuoka = gdf_shizuoka.reset_index(drop=True)

In [14]:
gdf_web_mercator = None

In [15]:
# 森林=1、建物用地=2、その他=3、に分類

gdf_shizuoka['category'] = 3
for i in range(len(gdf_shizuoka)):
  if gdf_shizuoka.loc[i, 'L03b_002'] == '0500':
    gdf_shizuoka.loc[i, 'category'] = 1
  elif gdf_shizuoka.loc[i, 'L03b_002'] == '0700':
    gdf_shizuoka.loc[i, 'category'] = 2
  else:
    gdf_shizuoka.loc[i, 'category'] = 3

In [16]:
# 面積を比較
print('forest: ', sum(gdf_shizuoka[gdf_shizuoka['category']==1].geometry.area))
print('city: ', sum(gdf_shizuoka[gdf_shizuoka['category']==2].geometry.area))

forest:  1720364569.1537557
city:  154963534.3680736


In [17]:
bounds = gdf_shizuoka.total_bounds  # (minx, miny, maxx, maxy)

# 出力するラスタの解像度を決定
pixel_size = 2
width = int((bounds[2] - bounds[0])/pixel_size)
height = int((bounds[3] - bounds[1])/pixel_size)

# 新しいラスタファイルのメタデータを作成
transform = rasterio.transform.from_origin(bounds[0], bounds[3], pixel_size, pixel_size)

# 5. `category`カラムをラスタ化: 属性値をラスタピクセル値に変換
# 各ポリゴンのジオメトリとその'category'値をリストにして渡す

shapes = [(geom, value) for geom, value in zip(gdf_shizuoka.geometry, gdf_shizuoka['category'])]

# ラスタ化
raster = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,  # 背景の値
    dtype=np.uint8  # valueバンド
)




In [ ]:
with rasterio.open('output.tif', 'w', driver='GTiff',
                   height=height, width=width,
                   count=2, dtype=np.uint8,  # 2バンド（valueとcategory）
                   crs=gdf_shizuoka.crs, transform=transform) as dst:
    # valueバンドを1バンド目として書き込み
    dst.write(raster, 1)
    # categoryバンドを2バンド目として書き込み
    dst.write(raster, 2)